### Description

Given a flexibility experiment with stored Pareto fronts, analyzes the Pareto fronts obtained. Also expects that the obtained Pareto fronts for each source material were evalated on each target material.

### Setup

In [ ]:
%matplotlib inline

import os, json, itertools, csv
import matplotlib.pyplot as plt
import numpy as np

from functools import reduce

from pymoo.indicators.hv import HV
from pymoo.indicators.gd import GD
from pymoo.indicators.igd import IGD

load_path = "../../../../flexbench-data/1-1_proof_of_concept_random_sampling/"
save_path = "../../../../flexbench-data/1-1_proof_of_concept_random_sampling_analysis/"

try:
    os.mkdir(save_path)
except FileExistsError:
    pass

materials = ["steel", "tungsten_alloy", "steel_dummy", "inconel_718"]
process_params = ["cutting_speed", "cutting_angle", "cutting_depth"]
objectives = ["production_time (log)", "tool_wear (log)", "Fc", "Ft"]
repetitions = 10

### Loading data

In [ ]:
def load_pareto_front(path):
    """Loads Pareto front from a given path."""
    
    with open(path) as f:
        pareto_front = json.load(f)
    
    return pareto_front

### Plots in the space of process parameters

In [ ]:
for rep in range(1,repetitions+1):

    # Initialize subplots.
    fig, axsVec = plt.subplots(nrows=2, ncols=2, figsize=(15,10), subplot_kw={'projection':'3d'})
    axsVec = [axs for axs in np.array(axsVec).flat]

    # For each material.
    for idx in range(len(materials)) :
    
        # Load data.
        material = materials[idx]
        data = load_pareto_front("%s%s/training/solutions/solutions_repetition_%d.json" % \
                                 (load_path,material,rep)
        data = np.array([x['solution'] for x in data])
    
        # Select subplot.
        axs = axsVec[idx]

        # Plot 3-D slice.
        im = axs.scatter(data[:,0], data[:,1], data[:,2])
        axs.set_xlabel(process_params[0])
        axs.set_ylabel(process_params[1])
        axs.set_zlabel(process_params[2])
        axs.set_title("%s\n(%d solutions in Pareto front)" % (material,len(data)))

    fig.savefig('%spareto_fronts_process_parameters_repetition_%d.png' % (save_path,rep))
    print('Plots saved in %s.' % save_path)

### Plots in the space of objectives (2d)

In [ ]:
for rep in range(1,repetitions+1):

    for material in materials:

        # Load data.
        data = load_pareto_front("%s%s/training/solutions/solutions_repetition_%d.json" % \
                                 (load_path,material,rep)
        data = np.array([x['performance'] for x in data])
        data[:,0] = np.log(data[:,0])
        data[:,1] = np.log(data[:,1])
    
        # Initialize subplots.
        fig, axsVec = plt.subplots(nrows=2, ncols=3, figsize=(20,10))
        fig.suptitle(material, fontsize=16)
        axsVec = [axs for axs in np.array(axsVec).flat]

        # All 2-D combinations of objectives.
        slices = [x for x in itertools.combinations(range(len(objectives)),2)]

        for idx in range(len(slices)) :

            # Select coords and subplot.
            coords = slices[idx]
            axs = axsVec[idx]

            # Plot 2-D slice.
            im = axs.scatter(x=data[:,coords[0]], y=data[:,coords[1]])
            axs.set_xlabel(objectives[coords[0]])
            axs.set_ylabel(objectives[coords[1]])
            axs.set_title('%s x %s' % (objectives[coords[0]], objectives[coords[1]]))

        fig.savefig('%spareto_fronts_objectives_2d_%s_repetition_%d.png' % (save_path, material, rep))
        print('Plots saved in %s.' % save_path)

### Plot in the space of objectives (2d), showing also the performance of the Pareto fronts from other materials.

In [ ]:
for rep in range(1,repetitions+1):

    for material in materials:

        data = {}
        
        # Load data for base material.
        data[material] = load_pareto_front("%s%s/training/solutions/solutions_repetition_%d.json" % \
                                           (load_path,material,rep))
        data[material] = np.array([x['performance'] for x in data[material]])
        data[material][:,0] = np.log(data[material][:,0])
        data[material][:,1] = np.log(data[material][:,1])
    
        # Load data for other materials.
        for material2 in materials:
            if material2 != material:
                data[material2] = load_pareto_front( \
                    "%spareto_fronts_evaluation/pareto_from_%s_evaluated_on_%s/solutions_%d.json" \
                    % (load_path,material2,material,rep))
                data[material2] = np.array([x['performance'] for x in data[material2]])
                data[material2][:,0] = np.log(data[material2][:,0])
                data[material2][:,1] = np.log(data[material2][:,1])
    
        # Initialize subplots.
        fig, axsVec = plt.subplots(nrows=2, ncols=3, figsize=(20,10))
        fig.suptitle(material, fontsize=16)
        axsVec = [axs for axs in np.array(axsVec).flat]

        # All 2-D combinations of objectives.
        slices = [x for x in itertools.combinations(range(len(objectives)),2)]

        for idx in range(len(slices)) :

            # Select coords and subplot.
            coords = slices[idx]
            axs = axsVec[idx]

            # Plot 2-D slice.
            for k in data.keys():
                im = axs.scatter(x=data[k][:,coords[0]], y=data[k][:,coords[1]], label=k)
            
            axs.set_xlabel(objectives[coords[0]])
            axs.set_ylabel(objectives[coords[1]])
            axs.set_title('%s x %s' % (objectives[coords[0]], objectives[coords[1]]))
            axs.legend()

        fig.savefig('%spareto_fronts_objectives_2d_%s_with_other_materials_repetition_%d.png' \
                    % (save_path, material, rep))
        print('Plots saved in %s.' % save_path)

### Table comparing Pareto fronts obtained for different materials when evaluated on the same material (mean over repetitions).

In [ ]:
def normalize(data, min_value, max_value):
    
    max_data = np.max(np.array([np.max(data[material], axis=0) for material in materials]), axis=0)
    min_data = np.min(np.array([np.min(data[material], axis=0) for material in materials]), axis=0)
    
    for i in range(len(objectives)):
    
        diff = max_value - min_value
        diff_data = max_data[i] - min_data[i]
    
        for material in materials:
            data[material][:,i] = (((data[material][:,i] - min_data[i])*diff)/diff_data) + min_value
    
    #for material in materials:
    #    print(np.min(data[material], axis=0))

            
# Initialize dictionary containing all measures obtained.
measures = {}
for material_source in materials:
    measures[material_source] = {}
    for material_target in materials:
        measures[material_source][material_target] = {}
        measures[material_source][material_target]["perc_feas"] = []
        measures[material_source][material_target]["hypervolume"] = []
        measures[material_source][material_target]["gen_dist"] = []
        measures[material_source][material_target]["inv_gen_dist"] = []
        measures[material_source][material_target]["min_tf"] = []

# Calculate measures for each repetition.
for rep in range(1,repetitions+1):

    # Initialize dictionary for holding data of all materials.
    data = {}

    for material_target in materials:
        
        # First, load data for all source materials.
        for material_source in materials:
            
            if material_source == material_target:
                data[material_source] = load_pareto_front( \
                    "%s%s/training/solutions/solutions_repetition_%d.json" % \
                    (load_path,material_source,rep))
            else:
                data[material_source] = load_pareto_front( \
                    "%spareto_fronts_evaluation/pareto_from_%s_evaluated_on_%s/solutions_%d.json" \
                    % (load_path,material_source,material_target,rep))
            
            # Add percentage of solutions that are feasible.
            p_feas = len([x for x in data[material_source] if np.all(x["feasibility"])])/len(data[material_source])
            measures[material_source][material_target]["perc_feas"].append(p_feas)
            
            #Process data.
            data[material_source] = np.array( \
                [x['performance'] for x in data[material_source] if np.all(x['feasibility'])]) #\ 
            data[material_source][:,0] = np.log(data[material_source][:,0])
            data[material_source][:,1] = np.log(data[material_source][:,1])
        
        # Normalize data.
        normalize(data, 0, 1)
        
        # Reference point for hypervolume: worst value for each objective.
        ref_point = np.max(np.array([np.max(data[material], axis=0) for material in materials]), axis=0)        
        
        # Calculate metrics.
        for material_source in materials:

            # Calculate hypervolume.
            function_hv = HV(ref_point=ref_point)
            hypervolume = function_hv(data[material_source])
                
            # Calculate gen. dist.
            function_gd = GD(data[material_target])
            gen_dist = function_gd(data[material_source])
                
            # Calculate inv. gen. dist.
            function_igd = IGD(data[material_target])
            inv_gen_dist = function_igd(data[material_source])
            
            # Calulate minimum trade-off.
            min_tf = np.min(np.sum(data[material_source], axis=1))
            
            # Add measures to dictionary.
            measures[material_source][material_target]["hypervolume"].append(hypervolume)
            measures[material_source][material_target]["gen_dist"].append(gen_dist)
            measures[material_source][material_target]["inv_gen_dist"].append(inv_gen_dist)
            measures[material_source][material_target]["min_tf"].append(min_tf)

def gen_table(mode):
            
    # Initialize table headers.
    table = []
    table.append([""]+reduce(lambda a,b:a+b, [["","",x,"",""] for x in materials]))
    table.append(["Source"]+["Perc. Feas.","Min. TF","Hypervolume","Gen. Dist.","Inv. Gen. Dist."]*len(materials))        

    for material_source in materials:
    
        # Initialize row.
        row = []
        row.append(material_source)
    
        for material_target in materials:
        
            # Select mode (mean or std).
            if mode == "mean":
                func = np.mean
            else:
                func = np.std
        
            # Calculate average over repetitions.
            perc_feas = func(np.array(measures[material_source][material_target]["perc_feas"]))
            hypervolume = func(np.array(measures[material_source][material_target]["hypervolume"]))
            gen_dist = func(np.array(measures[material_source][material_target]["gen_dist"]))
            inv_gen_dist = func(np.array(measures[material_source][material_target]["inv_gen_dist"]))
            min_tf = func(np.array(measures[material_source][material_target]["min_tf"]))
        
            # Add values to row.
            row.extend(["%.2f"%perc_feas,"%.4f"%min_tf,"%.4f"%hypervolume,"%.4f"%gen_dist,"%.4f"%inv_gen_dist])

        # Add row to table.
        table.append(row)        

    # Save table.
    with open("%stable_comparison_pareto_fronts_%s.csv"%(save_path,mode), "w", newline="") as f:
        writer = csv.writer(f, delimiter=",")
        writer.writerows(table)
        print("Table saved in %s." % save_path)

gen_table("mean")
gen_table("std")

### Table with selected solutions via minimum trade-off.

In [ ]:
# Initialize table.
table = [["","Process Parameters","","","Objectives","","",""],
         ["Task+Repetition"]+process_params+objectives]

# For each repetition.
for rep in range(1,repetitions+1):
    
    # For each source material.
    for material_target in materials:
        
        # For each target material.
        for material_source in materials:
            
            # Load data.
            if material_source == material_target:
                data = load_pareto_front("%s%s/training/solutions/solutions_repetition_%d.json" % \
                                         (load_path,material_source,rep))
            else:
                data = load_pareto_front( \
                    "%spareto_fronts_evaluation/pareto_from_%s_evaluated_on_%s/solutions_%d.json" \
                    % (load_path,material_source,material_target,rep))
            
            # Extract process and objectives data.
            data_process = [x['solution'] for x in data if np.all(x['feasibility'])]
            data_objectives = np.array([x['performance'] for x in data if np.all(x['feasibility'])])
            data_objectives[:,0] = np.log(data_objectives[:,0])
            data_objectives[:,1] = np.log(data_objectives[:,1])

            # Select solution with minimum trade-off.
            min_tf_idx = np.argmin(np.sum(data_objectives, axis=1))
            min_tf_process = data_process[min_tf_idx]
            min_tf_objectives = data_objectives[min_tf_idx]
            
            # Add row to table.
            if material_source == material_target:
                task_name = "%s_rep_%d" % (material_target,rep)
            else:
                task_name = "%s_on_%s_%d" % (material_source,material_target,rep)
            
            table.append([task_name]+[str(x) for x in min_tf_process]+[str(x) for x in min_tf_objectives])

# Save table.
with open(save_path+"table_selected_solutions_min_tf.csv", "w", newline="") as f:
    writer = csv.writer(f, delimiter=",")
    writer.writerows(table)
    print("Table saved in %s." % save_path)